什麼是執行緒：執行緒就是將所有工作分為N段，利用N個threading來做，同一個時間運行多個程序（在同個腳本）。
每一個def()就是一個程序
但因為Python具有Global interpreter locker，所以每一個時間其實都只有一個執行緒在執行而已

### Part I - 執行緒基本指令與操作

In [1]:
import threading

In [9]:
# 定義要給其他執行緒（線程）處理的程序
def thread_job():
    print("This is an added thread, number is %s" %threading.current_thread())

In [10]:
def main():
    added_thread = threading.Thread(target = thread_job)
    added_thread.start()

In [11]:
if __name__ == "__main__":
    main()

This is an added thread, number is <Thread(Thread-5, started 123145341964288)>


In [12]:
# 目前有幾個執行緒
print(threading.active_count())

# 這些執行緒名稱
print(threading.enumerate())

# 目前的程序是由哪些執行緒執行
print(threading.current_thread())

5
[<_MainThread(MainThread, started 4485338560)>, <Thread(Thread-2, started daemon 123145319870464)>, <Heartbeat(Thread-3, started daemon 123145325125632)>, <HistorySavingThread(IPythonHistorySavingThread, started 123145331453952)>, <ParentPollerUnix(Thread-1, started daemon 123145336709120)>]
<_MainThread(MainThread, started 4485338560)>


### Part II - join 功能：等執行緒執行完後，才執行的動作

In [21]:
import time

In [22]:
def T1_job():
    print("T1 start")
    for i in range(10):
        time.sleep(0.1)
    print("T1 finish!")

In [23]:
def T2_job():
    print("T2 start")
    print("T2 finish!")

In [24]:
def main():
    T1 = threading.Thread(target = T1_job)
    T2 = threading.Thread(target = T2_job)
    T1.start()
    T2.start()
    
    T1.join()
    T2.join()
    
    print("All done")

In [25]:
if __name__ == "__main__":
    main()

T1 start
T2 start
T2 finish!
T1 finish!
All done


### Part III - Queue:作為執行緒之返回值

In [31]:
from queue import Queue

In [32]:
def job(data, q):
    
    for i in range(len(data)):
        data[i] = data[i]**2
        
    return q.put(data)

In [33]:
def main(data):
    
    q = Queue()
    threads = []
    
    for i in range(4):
        thread = threading.Thread(target=job, args=(data[i], q))
        thread.start()
        threads.append(thread)
        
    for thread in threads:
        thread.join()
        
    results = []
    
    for _ in range(4):
        results.append(q.get())
        
    print(results)

In [34]:
if __name__ == "__main__":
    data = [[1,2,3,4], [2,3,4,5], [4,5,6,7], [3,4,5,6]]
    main(data)

[[1, 4, 9, 16], [4, 9, 16, 25], [16, 25, 36, 49], [9, 16, 25, 36]]


### Part IV - 多執行緒Python並非把所有任務平均交給多執行緒
Global Interpreter Lock (GIL): 同步執行緒的一種機制，使得任何時刻僅有一個執行緒在執行

In [38]:
import copy

In [37]:
def job(data, q):
    res = sum(data)
    q.put(res)

In [39]:
def multiThreading(data):
    q = Queue()
    threads = []
    
    for i in range(4):
        t = threading.Thread(target=job, args=(copy.copy(data), q), name = "T%s"%(i))
        t.start()
        threads.append(t)
        
    for thread in threads:
        thread.join()
        
    sum = 0
    for _ in range(4):
        sum += q.get()
    
    print(sum)

In [40]:
def normal(data):
    total = sum(data)
    print(total)

In [43]:
if __name__ == "__main__":
    
    data = list(range(1000000))
    start_time = time.time()
    normal(data*4)
    print("Normal: %f" %(time.time() - start_time))
    
    start_time = time.time()
    multiThreading(data)
    print("Multithreading: %f" %(time.time() - start_time))

1999998000000
Normal: 0.054944
1999998000000
Multithreading: 0.057185


### Part V - Lock:鎖住目前執行緒，把所有資源都拿來先執行此工作

In [48]:
def job1():
    global A, lock
    
    lock.acquire()
    for _ in range(10):
        A += 1
    print("job 1: ",A)
    lock.release()

In [49]:
def job2():
    global A, lock
    
    lock.acquire()
    for _ in range(10):
        A += 10
    print("job 2: ",A)
    lock.release()

In [56]:
if __name__ == "__main__":
    lock = threading.Lock()
    A = 0
    
    t1 = threading.Thread(target=job1, name = "t1")
    t2 = threading.Thread(target=job2, name = "t2")
    
    t1.start()
    t2.start()
    
    t1.join()
    t2.join()

job 1:  10
job 2:  110
